In [1]:
import numpy as np
import processingIO as pio

from pathlib import Path
from dataUtilities import filterData
from forceClass import Forces

In [27]:
data_location = r'/home/mco143/Documents/Current_OpenFOAM_Simulations/gridInd_Ebrahim_constant/'

solver_paths = pio.get_files(data_location, 'log.solver')
solver_paths.sort()

AttributeError: 'list' object has no attribute 'parts'

In [35]:
tmp = []

timeDict = {'totalTime':[], 'parentCase':[], 'specificCase':[]}

for solverFile in solver_paths:
    timeDict['parentCase'].append(solverFile.parts[-4])
    timeDict['specificCase'].append(solverFile.parts[-3])
    with open(solverFile, 'r') as f:
        for line in f:
            if 'ExecutionTime' in line:
                tmp.append(line.split()[2])

    timeDict['totalTime'].append(tmp[-1])      

In [37]:
timeDict

{'totalTime': ['401.19',
  '181.06',
  '67.86',
  '36.44',
  '28.68',
  '26.98',
  '28.04',
  '27.11',
  '28.32',
  '7805.1',
  '7067.29',
  '1259.34',
  '851.62',
  '694.23',
  '616.58',
  '595.76',
  '588.61',
  '566.23',
  '12874.55',
  '15129.7',
  '6882.9',
  '3224.24',
  '2093.71',
  '1729.04',
  '1541.78',
  '1558.67',
  '1494.32'],
 'parentCase': ['Ebrahim_constant1',
  'Ebrahim_constant1',
  'Ebrahim_constant1',
  'Ebrahim_constant1',
  'Ebrahim_constant1',
  'Ebrahim_constant1',
  'Ebrahim_constant1',
  'Ebrahim_constant1',
  'Ebrahim_constant1',
  'Ebrahim_constant2',
  'Ebrahim_constant2',
  'Ebrahim_constant2',
  'Ebrahim_constant2',
  'Ebrahim_constant2',
  'Ebrahim_constant2',
  'Ebrahim_constant2',
  'Ebrahim_constant2',
  'Ebrahim_constant2',
  'Ebrahim_constant3',
  'Ebrahim_constant3',
  'Ebrahim_constant3',
  'Ebrahim_constant3',
  'Ebrahim_constant3',
  'Ebrahim_constant3',
  'Ebrahim_constant3',
  'Ebrahim_constant3',
  'Ebrahim_constant3'],
 'specificCase': ['st0